In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import joblib

In [ ]:
!pip install shap -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 538.2/538.2 kB 7.0 MB/s eta 0:00:00


In [ ]:
!pip install LIME -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install --upgrade matplotlib -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 19.3 MB/s eta 0:00:00


In [ ]:
# %matplotlib inline
# %matplotlib notebook
import pandas as pd
import numpy as np
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, f1_score, roc_curve, precision_score, recall_score, accuracy_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import StratifiedGroupKFold
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
# import shap
# shap.initjs()
# import lime
# import lime.lime_tabular
import matplotlib.pyplot as plt

In [ ]:
def convert_feature_to_label(df, col):
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    return df

def convert_feature_to_one_hot(df, col):
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    oh = OneHotEncoder(handle_unknown='ignore')
    df_oh = pd.DataFrame(oh.fit_transform(df[[col]]).toarray())
    df = df.join(df_oh)

    rename_dict = {}

    for i, name in zip(df_oh.columns, le.classes_):
        rename_dict[i] = str(col) + '_' + str(name)

    df = df.rename(columns=rename_dict)
    df.drop(col, axis = 1, inplace = True)

    return df

def scale_feature(col, method='std'):
    if method == 'std':
        std = np.std(col)
        mean = np.mean(col)
        return (col - mean) / std, mean, std
    elif method =='minmax':
        min = np.min(col)
        max = np.max(col)
        return (col - min) / max

def calc_metrics(y_true, y_pred, metrics=[]):
    out = []
    #y_pred = y_pred >= 0.8357
    for met in metrics:
        if not met == roc_auc_score:
            out.append(met(y_true, y_pred.argmax(1)))
        else:
            out.append(met(y_true, np.max(y_pred, 1)))

    return out

def calc_class_weights(df, target='sga', type='log'):
    if type == 'log':
        valid_class = df[target].value_counts().sort_index().values
        # 1 / (1+ln(x))
        class_weights = 1./np.log1p(valid_class)
        # Normalize class_weights and multiply with number of classes
        class_weights = class_weights / class_weights.sum() * 2

    elif type == 'normal':
        classes = df[target].value_counts().sort_index().values
        class_weights = classes/df.shape[0]
        class_weights = 1 - class_weights
        class_weights = class_weights / class_weights.sum() * 2

    else:
        class_weights = None

    return class_weights

def ohe(df):
    # OHE for categorical data
    #for col in ['cord', 'presentation', 'placenta_site', 'hypertension', 'diabetes']:
    for col in ['hypertension', 'diabetes']:
        df = convert_feature_to_one_hot(df, col)
    for col in ['gender', 'smoking']:
        df = convert_feature_to_label(df, col)

    # OHE for ordinal data
    df = df.replace({'oligohydramnios' : 0, 'normal' : 1, 'polyhydramnios' : 2})
    return df

def ohe_tri3(df):
    # OHE for categorical data
    for col in ['presentation', 'placenta_site', 'hypertension', 'diabetes']:
        df = convert_feature_to_one_hot(df, col)
    for col in ['gender', 'smoking']:
        df = convert_feature_to_label(df, col)

    # OHE for ordinal data
    df = df.replace({'oligohydramnios' : 0, 'normal' : 1, 'polyhydramnios' : 2})
    return df

def data_impute(df):
    # Iterative data imputation
    imputer = IterativeImputer(random_state = 123)
    imputed = imputer.fit_transform(df)
    return pd.DataFrame(imputed, columns = df.columns)

def sgkf(df, label = 'sga'):
    sgkf = StratifiedGroupKFold(shuffle=True, random_state=123, n_splits = 5)

    df['fold'] = -1

    for fold_num, (_, val_idx) in enumerate(sgkf.split(df, df[label], groups=df.id)):
        df.loc[val_idx, 'fold'] = fold_num

    # tri2[tri2['fold'] == 0].sga.value_counts()
    return df.drop('id', axis=1)

# Tri3

In [ ]:
def lr(df, label = 'sga'):

    train_cols = df.columns
    train_cols = train_cols.drop([label, 'fold'])

    oof_acc = []
    oof_roc_auc = []
    oof_f1 = []
    oof_prec = []
    oof_rec = []
    oof_thresh = []
    temp = None

    #class_weights = calc_class_weights(df, target= label, type='normal')

    for fold in range(5):
        #rfc = RandomForestClassifier(n_estimators = 56, criterion = 'log_loss', max_depth = 3, min_samples_split = 0.27, min_samples_leaf = 8, bootstrap = False, ccp_alpha = 0.05, max_features = None, min_impurity_decrease = 0.0332, random_state = 123)
        rfc = RandomForestClassifier(max_features = None, bootstrap = False, n_estimators = 1, max_depth = 1, criterion = "entropy", min_impurity_decrease = 0.1, verbose = 0, random_state = 123, min_samples_leaf = 1, ccp_alpha = 0.2)
        ros = RandomOverSampler(random_state = 196)
        train_df = df[df['fold'] != fold].reset_index(drop=True)
        train_X, train_Y = ros.fit_resample(train_df[train_cols], train_df[label])
        val_df = df[df['fold'] == fold].reset_index(drop=True)

        #sample_weights = [class_weights[int(x)] for x in train_Y]

        rfc.fit(train_X, train_Y)

        out = rfc.predict_proba(val_df[train_cols])
        acc, roc_auc, f1, prec, rec = calc_metrics(val_df[label], out, metrics=[balanced_accuracy_score, roc_auc_score, f1_score, precision_score, recall_score])
        out2 = rfc.predict(val_df[train_cols])

        oof_acc.append(acc)
        oof_roc_auc.append(roc_auc)
        oof_f1.append(f1)
        oof_prec.append(prec)
        oof_rec.append(rec)

        print("********************")
        print(f"Fold: {fold}")
        print(f"Balanced Accuracy: {acc:.4f}")
        print(f"ROC AUC: {roc_auc:.4f}")
        print(f"F1: {f1:.4f}")
        print(f"Precision: {prec:.4f}")
        print(f"Recall: {rec:.4f}")
        joblib.dump(rfc, f'tri3_rfc_fold_{fold}.joblib')

        if fold == 2:
          print(val_df[train_cols].columns)
          val_df.to_csv("/content/tri3_df_fold_2.csv")



    print(f'OOF Balanced Accuracy: {np.mean(oof_acc):.4f} (±{np.std(oof_acc):.4f})')
    print(f'OOF ROC AUC Score: {np.mean(oof_roc_auc):.4f} (±{np.std(oof_roc_auc):.4f})')
    print(f'OOF F1 Score: {np.mean(oof_f1):.4f} (±{np.std(oof_f1):.4f})')
    print(f'OOF Prec Score: {np.mean(oof_prec):.4f} (±{np.std(oof_prec):.4f})')
    print(f'OOF Recall Score: {np.mean(oof_rec):.4f} (±{np.std(oof_rec):.4f})')
    return temp


tri3 = pd.read_csv('/content/cyf_clean_tri3.csv')
tri3.drop(['lbw', 'sga'], axis = 1, inplace = True)
#continuous_col = ['ac', 'bpd', 'efw_centile', 'efw', 'fl', 'ga', 'hc', 'mother_age_at_start_date', 'mother_height', 'mother_weight']
#tri3[continuous_col] = scale_feature(tri3[continuous_col], method='std')
#tri3 = tri3[['ac', 'af', 'bpd', 'efw_centile', 'efw', 'fl', 'ga', 'hc', 'mother_age_at_start_date', 'mother_height', 'mother_weight', 'smoking', 'gender', 'presentation', 'placenta_site', 'hypertension', 'diabetes']]
tri3 = ohe_tri3(tri3)
tri3 = data_impute(tri3)
tri3 = sgkf(tri3, label = 'status_change')
lr(tri3, label = 'status_change')


FileNotFoundError: [Errno 2] No such file or directory: '/content/cyf_clean_tri3.csv'

# Tri2

In [ ]:
def rf(df, label = 'sga'):

    train_cols = df.columns
    train_cols = train_cols.drop([label, 'fold', 'cur_sga', 'sga'])

    oof_acc = []
    oof_roc_auc = []
    oof_f1 = []
    oof_prec = []
    oof_rec = []
    oof_thresh = []
    temp = None

    class_weights = calc_class_weights(df, target= label, type='normal')



    for fold in range(5):

      ros = RandomOverSampler(random_state = 317)
      rfc = RandomForestClassifier(n_estimators = 56, criterion = 'log_loss', max_depth = 3, min_samples_split = 0.27, min_samples_leaf = 8, bootstrap = False, ccp_alpha = 0.05, max_features = None, min_impurity_decrease = 0.0332, random_state = 123)
      train_df = df[df['fold'] != fold].reset_index(drop=True)
      train_X, train_Y = ros.fit_resample(train_df[train_cols], train_df[label])
      val_df = df[df['fold'] == fold].reset_index(drop=True)
      tmp = val_df[['sga', 'cur_sga', label]]
      val_df = val_df.drop(['sga', 'cur_sga'], axis=1)
      sample_weights = [class_weights[int(x)] for x in train_Y]

      rfc.fit(train_X, train_Y)

      out = rfc.predict_proba(val_df[train_cols])
      acc, roc_auc, f1, prec, rec = calc_metrics(val_df[label], out, metrics=[accuracy_score, roc_auc_score, f1_score, precision_score, recall_score])
      out2 = rfc.predict_proba(val_df[train_cols])
      out2 = out2.argmax(1)
      # print(out2)
      cur_sga = tmp['sga']
      # cur_sga2 = np.logical_xor(tmp[label], tmp['sga']).astype(int)
      # check = all(np.logical_not(np.logical_xor(cur_sga, cur_sga2)).astype(int))
      # print(check)
      out2 = np.logical_xor(out2, cur_sga.to_numpy())
      print(out2.astype(int))

      oof_acc.append(acc)
      oof_roc_auc.append(roc_auc)
      oof_f1.append(f1)
      oof_prec.append(prec)
      oof_rec.append(rec)
      joblib.dump(rfc, f'tri2_rfc_fold_{fold}.joblib')

      print("********************")
      print(f"Fold: {fold}")
      print(f"Balanced Accuracy: {acc:.4f}")
      print(f"ROC AUC: {roc_auc:.4f}")
      print(f"F1: {f1:.4f}")
      print(f"Precision: {prec:.4f}")
      print(f"Recall: {rec:.4f}")
      print(f"Real SGA Balanced Accuracy: {accuracy_score(out2, tmp['sga']):.4f}")
      if fold == 0:
          val_df['sga'] = tmp['sga']
          val_df['cur_sga'] = tmp['cur_sga']
          val_df['pred_status_change'] = out.argmax(1)
          val_df['pred_sga'] = out2
          val_df.to_csv("/content/tri2_df_fold_0.csv")


    # 1. Print accuracy of 5 folds
    # 2. Save cross validation data and test on dashboard
    # 3.

    # Calculate SHAP values after training on all folds
    # (you may want to use a subset of your data for better runtime)
    # shap_explainer = shap.TreeExplainer(rf)
    # all_data = df[train_cols]  # or use a subset if your dataset is large
    # shap_values = shap_explainer.shap_values(all_data)

    # # Choose an instance for which you want to generate SHAP force plot
    # instance_idx = 0  # Choose the index of the instance

    # %matplotlib inline
    # # Generate SHAP force plot
    # shap.force_plot(
    #     shap_explainer.expected_value[1],  # Shap values for the positive class
    #     shap_values[1][instance_idx, :],
    #     features=all_data.iloc[instance_idx, :],
    #     feature_names=train_cols
    # )

    # plt.show()

    # # Use SHAP values for interpretation or visualization
    # # For example, you can print a summary plot
    #shap.summary_plot(shap_values[1], features=all_data, feature_names=train_cols)
    # #shap.plots.waterfall(shap_values)

    # Calculate LIME explanations after training on all folds
    # explainer = lime.lime_tabular.LimeTabularExplainer(
    #     training_data=train_X.values,  # Use training data for explanation
    #     mode='classification',
    #     feature_names=train_cols,
    #     class_names=['Not Positive', 'Positive'],
    #     discretize_continuous=True  # Set to False if your data is not continuous
    # )

    # # Choose an instance for which you want to explain the prediction
    # instance_idx = 10  # Choose the index of the instance you want to explain
    # instance = val_df[train_cols].iloc[[instance_idx]].values

    # # Get a LIME explanation for the Random Forest prediction
    # lime_explanation = explainer.explain_instance(
    #     instance[0],
    #     rf.predict_proba,
    #     num_features=len(train_cols),
    #     top_labels=1,
    # )

    # # Display the LIME explanation
    # lime_explanation.show_in_notebook()

    # #plt = lime_explanation.as_pyplot_figure()

    print(f'OOF Balanced Accuracy: {np.mean(oof_acc):.4f} (±{np.std(oof_acc):.4f})')
    print(f'OOF ROC AUC Score: {np.mean(oof_roc_auc):.4f} (±{np.std(oof_roc_auc):.4f})')
    print(f'OOF F1 Score: {np.mean(oof_f1):.4f} (±{np.std(oof_f1):.4f})')
    print(f'OOF Prec Score: {np.mean(oof_prec):.4f} (±{np.std(oof_prec):.4f})')
    print(f'OOF Recall Score: {np.mean(oof_rec):.4f} (±{np.std(oof_rec):.4f})')

    # joblib.dump(rfc, '/content/drive/MyDrive/SGA/withBiomarker/trained_model.pkl')

    return temp


# tri2 = pd.read_csv('/content/drive/MyDrive/SGA/cyf_clean_tri2.csv')
tri2 = pd.read_csv('/content/drive/MyDrive/SGA/cyf_clean_tri2.csv')
# tri2.drop(['sga', 'lbw', 'cur_sga'], axis = 1, inplace = True)
continuous_col = ['ac', 'bpd', 'cm', 'efw_centile', 'efw', 'fl', 'ga', 'hc', 'hl', 'tcd', 'mother_age_at_start_date', 'mother_height', 'mother_weight']
ori_tri2 = tri2.copy()
# for c in continuous_col:
#   tri2[c], _, _ = scale_feature(tri2[c], method='std')

# means.to_csv("tri2_means.csv")
# stds.to_csv("tri2_stds.csv")
tri2 = tri2[['ac', 'bpd', 'efw_centile', 'efw', 'fl', 'ga', 'hc', 'mother_age_at_start_date', 'mother_height', 'mother_weight', 'smoking', 'gender', 'hypertension', 'diabetes', 'status_change', 'id', 'sga', 'cur_sga']]
tri2 = ohe(tri2)
tri2 = data_impute(tri2)
tri2 = sgkf(tri2, label = 'status_change')
tri2.info()
rf(tri2, label = 'status_change')
# joblib.dump(rf, 'tri2_rf.joblib')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 602 entries, 0 to 601
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ac                        602 non-null    float64
 1   bpd                       602 non-null    float64
 2   efw_centile               602 non-null    float64
 3   efw                       602 non-null    float64
 4   fl                        602 non-null    float64
 5   ga                        602 non-null    float64
 6   hc                        602 non-null    float64
 7   mother_age_at_start_date  602 non-null    float64
 8   mother_height             602 non-null    float64
 9   mother_weight             602 non-null    float64
 10  smoking                   602 non-null    float64
 11  gender                    602 non-null    float64
 12  status_change             602 non-null    float64
 13  sga                       602 non-null    float64
 14  cur_sga   

In [ ]:
plt.hist(tri2['bpd'])
plt.hist(ori_tri2['bpd'])

tmp = (ori_tri2['bpd'] - np.mean(ori_tri2['bpd'])) / np.std(ori_tri2['bpd'])
plt.hist(tmp)
print(tmp.min(), tmp.max())
print(tri2['bpd'].min(), tri2['bpd'].max())
plt.legend()
plt.show()

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/SGA/withBiomarker/with_biomarker_test_data.csv', usecols=lambda column: column != 'Unnamed: 0')
test.drop(['sga', 'lbw', '2_cur_sga', 'diabetes', 'hypertension'], axis = 1, inplace = True)
continuous_col = ['2_ac', '2_bpd', '2_efw_centile', '2_efw', '2_fl', '2_ga', '2_hc', '1_m_age', '1_m_height', '1_m_weight']
test[continuous_col] = scale_feature(test[continuous_col], method='std')
test['sc'] = test['sc'].astype(int)
test.rename(columns = {'2_ac' : 'ac',
                        '2_bpd' : 'bpd',
                        '2_efw_centile' : 'efw_centile',
                        '2_efw' : 'efw',
                        '2_fl' : 'fl',
                        '2_ga' : 'ga',
                        '2_hc' : 'hc',
                        '1_m_age' : 'mother_age_at_start_date',
                        '1_m_height' : 'mother_height',
                        '1_m_weight' : 'mother_weight',
                        'sc' : 'status_change'}, inplace = True)
test['diabetes_0.0'] = 0
test['diabetes_1.0'] = 0
test['diabetes_2.0'] = 0
test['hypertension_0.0'] = 0
test['hypertension_1.0'] = 0
test['hypertension_2.0'] = 0
test['smoking'] = 0
test = test[['ac', 'bpd', 'efw_centile', 'efw', 'fl', 'ga', 'hc', 'mother_age_at_start_date', 'mother_height', 'mother_weight', 'smoking', 'status_change', 'gender', 'hypertension_0.0', 'hypertension_1.0', 'hypertension_2.0', 'diabetes_0.0', 'diabetes_1.0', 'diabetes_2.0']]
test.info()

# Test

In [ ]:
trained_model = joblib.load('/content/drive/MyDrive/SGA/withBiomarker/trained_model.pkl')
X_test = test.drop('status_change', axis = 1)  # Assuming 'sc' is the target variable
y_true = test['status_change']
predictions = trained_model.predict(X_test)
balanced_accuracy = balanced_accuracy_score(y_true, predictions)
precision = precision_score(y_true, predictions)
recall = recall_score(y_true, predictions)
f1 = f1_score(y_true, predictions)
roc_auc = roc_auc_score(y_true, predictions)
test['prediction'] = predictions
test['status_change'] = y_true

# Print or display the evaluation results
print(f'Accuracy: {balanced_accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-score: {f1:.2f}')
print(f'ROC AUC: {roc_auc:.2f}')

In [ ]:
test.info()

In [ ]:
test.to_csv('/content/drive/MyDrive/SGA/withBiomarker/result_of_test_data_on_trained_model.csv')

In [ ]:
# def rf(df, label = 'sga'):

#     train_cols = df.columns
#     train_cols = train_cols.drop([label, 'fold'])

#     oof_acc = []
#     oof_roc_auc = []
#     oof_f1 = []
#     oof_prec = []
#     oof_rec = []
#     oof_thresh = []
#     temp = None

#     class_weights = calc_class_weights(df, target= label, type='normal')

#     rf = LogisticRegression(class_weight = 'balanced')

#     for fold in range(5):

#       ros = RandomOverSampler(random_state = 317)

#       train_df = df[df['fold'] != fold].reset_index(drop=True)
#       train_X, train_Y = ros.fit_resample(train_df[train_cols], train_df[label])
#       val_df = df[df['fold'] == fold].reset_index(drop=True)
#       sample_weights = [class_weights[int(x)] for x in train_Y]

#       rf.fit(train_X, train_Y)

#       out = rf.predict_proba(val_df[train_cols])
#       acc, roc_auc, f1, prec, rec = calc_metrics(val_df[label], out, metrics=[balanced_accuracy_score, roc_auc_score, f1_score, precision_score, recall_score])
#       out2 = rf.predict(val_df[train_cols])

#       oof_acc.append(acc)
#       oof_roc_auc.append(roc_auc)
#       oof_f1.append(f1)
#       oof_prec.append(prec)
#       oof_rec.append(rec)

#     # Calculate SHAP values after training on all folds
#     # (you may want to use a subset of your data for better runtime)
#     # shap_explainer = shap.TreeExplainer(rf)
#     # all_data = df[train_cols]  # or use a subset if your dataset is large
#     # shap_values = shap_explainer.shap_values(all_data)

#     # # Choose an instance for which you want to generate SHAP force plot
#     # instance_idx = 0  # Choose the index of the instance

#     # %matplotlib inline
#     # # Generate SHAP force plot
#     # shap.force_plot(
#     #     shap_explainer.expected_value[1],  # Shap values for the positive class
#     #     shap_values[1][instance_idx, :],
#     #     features=all_data.iloc[instance_idx, :],
#     #     feature_names=train_cols
#     # )

#     # plt.show()

#     # # Use SHAP values for interpretation or visualization
#     # # For example, you can print a summary plot
#     #shap.summary_plot(shap_values[1], features=all_data, feature_names=train_cols)
#     # #shap.plots.waterfall(shap_values)

#     # Calculate LIME explanations after training on all folds
#     # explainer = lime.lime_tabular.LimeTabularExplainer(
#     #     training_data=train_X.values,  # Use training data for explanation
#     #     mode='classification',
#     #     feature_names=train_cols,
#     #     class_names=['Not Positive', 'Positive'],
#     #     discretize_continuous=True  # Set to False if your data is not continuous
#     # )

#     # # Choose an instance for which you want to explain the prediction
#     # instance_idx = 10  # Choose the index of the instance you want to explain
#     # instance = val_df[train_cols].iloc[[instance_idx]].values

#     # # Get a LIME explanation for the Random Forest prediction
#     # lime_explanation = explainer.explain_instance(
#     #     instance[0],
#     #     rf.predict_proba,
#     #     num_features=len(train_cols),
#     #     top_labels=1,
#     # )

#     # # Display the LIME explanation
#     # lime_explanation.show_in_notebook()

#     # #plt = lime_explanation.as_pyplot_figure()

#     print(f'OOF Balanced Accuracy: {np.mean(oof_acc):.4f} (±{np.std(oof_acc):.4f})')
#     print(f'OOF ROC AUC Score: {np.mean(oof_roc_auc):.4f} (±{np.std(oof_roc_auc):.4f})')
#     print(f'OOF F1 Score: {np.mean(oof_f1):.4f} (±{np.std(oof_f1):.4f})')
#     print(f'OOF Prec Score: {np.mean(oof_prec):.4f} (±{np.std(oof_prec):.4f})')
#     print(f'OOF Recall Score: {np.mean(oof_rec):.4f} (±{np.std(oof_rec):.4f})')

#     return temp


# tri2 = pd.read_csv('/content/drive/MyDrive/SGA/cyf_clean_tri2.csv')
# tri2.drop(['sga', 'lbw', 'cur_sga'], axis = 1, inplace = True)
# continuous_col = ['ac', 'bpd', 'cm', 'efw_centile', 'efw', 'fl', 'ga', 'hc', 'hl', 'tcd', 'mother_age_at_start_date', 'mother_height', 'mother_weight']
# tri2[continuous_col] = scale_feature(tri2[continuous_col], method='std')
# tri2 = tri2[['ac', 'bpd', 'efw_centile', 'efw', 'fl', 'ga', 'hc', 'mother_age_at_start_date', 'mother_height', 'mother_weight', 'smoking', 'gender', 'hypertension', 'diabetes', 'status_change', 'id']]
# tri2 = ohe(tri2)
# tri2 = data_impute(tri2)
# tri2 = sgkf(tri2, label = 'status_change')
# rf(tri2, label = 'status_change')

In [ ]:
tri2 = pd.read_csv('/content/drive/MyDrive/SGA/cyf_clean_tri2.csv')
tri2.info()

In [ ]:
import random
import math
import numpy as np
cur_sga = np.array([random.randint(0, 1) for _ in range(10)])
sga = np.array([random.randint(0, 1) for _ in range(10)])
pred_sc = np.array([random.randint(0, 1) for _ in range(10)])
sc = np.logical_xor(cur_sga, sga).astype(int)
correct_sc = np.logical_not(np.logical_xor(pred_sc, sc)).astype(int)
pred_sga = np.logical_xor(pred_sc, cur_sga).astype(int)
correct_sga = np.logical_not(np.logical_xor(pred_sga, sga)).astype(int)
print(cur_sga)
print(sga)
print(pred_sc)
print(sc)
print(correct_sc)
print(pred_sga)
print(correct_sga)

In [2]:
import numpy as np
from sklearn.metrics import accuracy_score, balanced_accuracy_score
cur_sga = '''0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0'''
cur_sga = np.array(cur_sga.split('. ')).astype(int)

sga = '''0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0'''
sga = np.array(sga.split('. ')).astype(int)
sc = np.logical_xor(cur_sga, sga)
ref_sc = '''0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0'''
ref_sc = np.array(ref_sc.split('. ')).astype(int)
pred_sc = '''0  0  0  0  0  0  0  0  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  1  0  0  0  0  0  0  1  1  0  0  0  0  0  0  1  0  1  0  1  0  0  0  0  1  1  0  0  0  0  0  1  1  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0'''
pred_sc = np.array(pred_sc.split('  ')).astype(int)
correct_sc = np.logical_not(np.logical_xor(pred_sc, sc)).astype(int)
pred_sga = np.logical_xor(pred_sc, cur_sga).astype(int)
#print(balanced_accuracy_score(pred_sc, sc))
print(accuracy_score(pred_sc, sc))
print(accuracy_score(pred_sga, sga))

0.9646017699115044
0.9646017699115044
